<a href="https://colab.research.google.com/github/alex-pakalniskis/gisc606-spring2023/blob/main/lab4/GISC606_L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Uncertainty in Remote Sensing Data

*Due April 22 before 9am Pacific*

This lab is an adaptation of Earth Lab's [Uncertainty and Remote Sensing Data](https://www.earthdatascience.org/courses/use-data-open-source-python/spatial-data-applications/lidar-remote-sensing-uncertainty/) textbook course module


## Part 0:Workspace setup

In [ ]:
# Download Lidar data and unzip it

# https://linux.die.net/man/1/wget
!wget https://ndownloader.figshare.com/files/12459464

#https://linux.die.net/man/1/unzip
!unzip 12459464 

In [ ]:
# Install python dependencies with pip

# pip: https://pypi.org/project/pip/
# Geopandas: https://geopandas.org/en/stable/
# Rioxarray: https://corteva.github.io/rioxarray/stable/
# Earthpy: https://earthpy.readthedocs.io/en/latest/index.html
# Rasterstats: https://pythonhosted.org/rasterstats/
!pip install geopandas rioxarray earthpy rasterstats

In [ ]:
# Import some libraries to visualize study site as a webmap

# Geopandas: https://geopandas.org/en/stable/
import geopandas as gpd

# Folium: https://python-visualization.github.io/folium/
import folium

In [ ]:
# Read the shapefile into memory

# https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html
site_boundary = gpd.read_file("california/neon-sjer-site/vector_data/SJER_plot_centroids.shp")

# Transform shapefile geometries to a new coordinate reference system

# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html
site_boundary_prj = site_boundary.to_crs("EPSG:4326")


# Instantiate a folium map

# https://python-visualization.github.io/folium/modules.html#module-folium.map
m = folium.Map(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
    )

# Add GeoJSON vector layer to the folium map

# https://python-visualization.github.io/folium/modules.html#folium.features.GeoJson
folium.GeoJson(data=site_boundary["geometry"]).add_to(m)

# Create a helper variable for prettier plotting
padding = 0.01

# Fit the map to a bounding box (with padding) around the GeoJSON vector layer

# https://python-visualization.github.io/folium/modules.html#folium.folium.Map.fit_bounds
m.fit_bounds(
    [
        [
            site_boundary_prj.total_bounds[1]-padding, 
            site_boundary_prj.total_bounds[0]-padding
         ],
        [
            site_boundary_prj.total_bounds[3]+8*padding, 
            site_boundary_prj.total_bounds[2]+8*padding
         ]
        ]
    )


# Plot the map
m

## Part 1: Extract Data from Raster

In [ ]:
# Import more Python libraries for data analysis and visualization

#https://docs.python.org/3/library/os.html
import os 
#https://matplotlib.org/stable/api/pyplot_summary.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt 
#https://seaborn.pydata.org/
import seaborn as sns 
#https://numpy.org/
import numpy as np 
#https://numpy.org/doc/stable/reference/maskedarray.generic.html
import numpy.ma as ma 
#https://pandas.pydata.org/
import pandas as pd 
#https://corteva.github.io/rioxarray/stable/
import rioxarray as rxr 
#https://rasterio.readthedocs.io/en/stable/api/rasterio.plot.html#rasterio.plot.plotting_extent
from rasterio.plot import plotting_extent 
#https://geopandas.org/en/stable/
import geopandas as gpd 
#https://pythonhosted.org/rasterstats/
import rasterstats as rs 
#https://earthpy.readthedocs.io/en/latest/
import earthpy as et 
#https://earthpy.readthedocs.io/en/latest/api/earthpy.plot.html
import earthpy.plot as ep 
#https://seaborn.pydata.org/generated/seaborn.set_style.html
sns.set_style("white") 
#https://seaborn.pydata.org/generated/seaborn.set.html
sns.set(font_scale=1.5) 

In [ ]:
# Download data (technically redundant but we'll do it anyways) and set working directory

#https://earthpy.readthedocs.io/en/latest/api/earthpy.io.html#earthpy.io.Data.get_data
data = et.data.get_data("spatial-vector-lidar") 
#https://docs.python.org/3/library/os.html#os.chdir, https://docs.python.org/3/library/os.path.html#os.path.join
os.chdir(
    os.path.join(
        et.io.HOME, 
        'earth-analytics', 
        'data'
        )
    ) 

In [ ]:
# Load & plot the data

#https://docs.python.org/3/library/os.path.html#os.path.join
sjer_lidar_chm_path = os.path.join(
    "spatial-vector-lidar",
    "california", 
    "neon-sjer-site",
    "2013", 
    "lidar", 
    "SJER_lidarCHM.tif"
    ) 
#https://docs.xarray.dev/en/stable/generated/xarray.open_rasterio.html
sjer_chm_data = rxr.open_rasterio(
    sjer_lidar_chm_path, 
    masked=True
    ).squeeze() 

In [ ]:
# Explore the data by plotting a histogram with earthpy

#https://earthpy.readthedocs.io/en/latest/api/earthpy.plot.html#earthpy.plot.hist
ax = ep.hist(
    sjer_chm_data.values,
    figsize=(8, 8),
    colors="purple",
    xlabel="Lidar Estimated Tree Height (m)",
    ylabel="Total Pixels",
    title="Distribution of Pixel Values \n Lidar Canopy Height Model"
    ) 
#https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.ticklabel_format.html
ax[1].ticklabel_format(
    useOffset=False,
    style='plain'
    ) 

In [ ]:
# View summary statistics of canopy height model
# Notice the mean value with 0's included in the data

#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html
print('Mean:', sjer_chm_data.mean().values) 
#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.max.html
print('Max:', sjer_chm_data.max().values) 
#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.min.html
print('Min:', sjer_chm_data.min().values) 

In [ ]:
# Set CHM values of 0 to NAN (no data or not a number) and view summary statistics of canopy height model after cleaning up the data

#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html
sjer_chm_data_no_zeros = sjer_chm_data.where(sjer_chm_data != 0, np.nan)
#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html
print('Mean:', sjer_chm_data_no_zeros.mean().values) 
#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.max.html
print('Max:', sjer_chm_data_no_zeros.max().values) 
#https://docs.xarray.dev/en/stable/generated/xarray.DataArray.min.html
print('Min:', sjer_chm_data_no_zeros.min().values) 

In [ ]:
# Explore the data by plotting a histogram with earthpy

#https://earthpy.readthedocs.io/en/latest/api/earthpy.plot.html#earthpy.plot.hist
ax = ep.hist(sjer_chm_data_no_zeros.values,
             figsize=(8, 8),
             colors="purple",
             xlabel="Lidar Estimated Tree Height (m)",
             ylabel="Total Pixels",
             title="Distribution of Pixel Values \n Lidar Canopy Height Model") 
#https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.ticklabel_format.html
ax[1].ticklabel_format(useOffset=False,
                       style='plain') 

In [ ]:
# Explore the insitu measurement data

#https://docs.python.org/3/library/os.path.html#os.path.join
sjer_centroids_path = os.path.join("spatial-vector-lidar",
                                   "california", 
                                   "neon-sjer-site",
                                   "vector_data", 
                                   "SJER_plot_centroids.shp") 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html
sjer_plots_points = gpd.read_file(sjer_centroids_path) 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html
sjer_plots_points.head() 


In [ ]:
# Plot the insitu measurement site data

#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
fig, ax = plt.subplots(figsize=(10, 10)) 

# We plot with the zeros in the data so the CHM can be better represented visually
#https://earthpy.readthedocs.io/en/latest/api/earthpy.plot.html#earthpy.plot.plot_bands
ep.plot_bands(sjer_chm_data,
              extent=plotting_extent(sjer_chm_data,
                                     sjer_chm_data.rio.transform()),  # Set spatial extent
              cmap='Greys',
              title="San Joachin Field Site \n Vegetation Plot Locations",
              scale=False,
              ax=ax) 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html
sjer_plots_points.plot(ax=ax,
                       marker='s',
                       markersize=45,
                       color='purple') 
#https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_axis_off.html
ax.set_axis_off() 
#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html
plt.show() 

In [ ]:
# Make a copy of the plot points for buffering

#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html
sjer_plots_poly = sjer_plots_points.copy() 

# Create a buffered polygon layer from your plot location points
# Buffer each point using a 20 meter circle radius
# and replace the point geometry with the new buffered geometry

#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html
sjer_plots_poly["geometry"] = sjer_plots_points.geometry.buffer(20) 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html
sjer_plots_poly.head() 

In [ ]:
# If the dir does not exist, create it

#https://docs.python.org/3/library/os.path.html#os.path.join
output_path = os.path.join("spatial-vector-lidar", 
                           "outputs") 
#https://docs.python.org/3/library/os.path.html#os.path.isdir
if not os.path.isdir(output_path): 
#https://docs.python.org/3/library/os.html#os.mkdir
    os.mkdir(output_path) 

# Export the buffered point layer as a shapefile to use in zonal stats

#https://docs.python.org/3/library/os.path.html#os.path.join
plot_buffer_path = os.path.join(output_path, 
                                "plot_buffer.shp") 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html
sjer_plots_poly.to_file(plot_buffer_path) 

In [ ]:
# Extract zonal stats

#https://pythonhosted.org/rasterstats/rasterstats.html#rasterstats.zonal_stats
sjer_tree_heights = rs.zonal_stats(plot_buffer_path,
                                   sjer_chm_data_no_zeros.values,
                                   nodata=-999,
                                   affine=sjer_chm_data_no_zeros.rio.transform(),
                                   geojson_out=True,
                                   copy_properties=True,
                                   stats="count min mean max median") 

# View object type

#https://docs.python.org/3/library/functions.html#type
type(sjer_tree_heights) 

In [ ]:
# Turn extracted data into a geopandas geodataframe

#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.from_features.html
sjer_lidar_height_df = gpd.GeoDataFrame.from_features(sjer_tree_heights) 
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html
sjer_lidar_height_df.head() 

In [ ]:
# Plot the geodataframe

fig, ax = plt.subplots(figsize=(10, 5))

#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html
ax.bar(sjer_lidar_height_df['Plot_ID'],
       sjer_lidar_height_df['max'],
       color="purple") 

ax.set(xlabel='Plot ID', 
       ylabel='Max Height',
       title='Maximum LIDAR Derived Tree Heights')
#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.setp.html
plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right') 
plt.show()

In [ ]:
# Extract zonal stats but retain the individual pixel values
sjer_tree_heights_ras = rs.zonal_stats(plot_buffer_path,
                                       sjer_chm_data_no_zeros.values,
                                       nodata=-999,
                                       affine=sjer_chm_data_no_zeros.rio.transform(),
                                       geojson_out=True,
                                       raster_out=True,
                                       copy_properties=True,
                                       stats="count min mean max median")
# Convert to geodataframe
sjer_lidar_height_df_ras = gpd.GeoDataFrame.from_features(
    sjer_tree_heights_ras)

# View subset of the dataframe
sjer_lidar_height_df_ras[["Plot_ID", 
                          "count", 
                          "geometry",
                          "mini_raster_affine", 
                          "mini_raster_array"]].head()

In [ ]:
# Display dataframe without subsetting columns
sjer_lidar_height_df_ras.head()

## Part 2: Compare Lidar to Measured Tree Height

In [ ]:
# Get list of sites
site_names = list(sjer_lidar_height_df_ras["Plot_ID"])

# Convert data in dataframe to a numpy array
#https://numpy.org/doc/stable/reference/generated/numpy.stack.html
arr = np.stack(sjer_lidar_height_df_ras['mini_raster_array']) 

# Plot using earthpy
ep.hist(arr,
        bins=[0, 5, 10, 15, 20, 25],
        cols=3,
        title=site_names, 
        figsize=(15, 30))

plt.show()

In [ ]:
# Import & view insitu (field measured) data
path_insitu = os.path.join("spatial-vector-lidar",
                           "california",
                           "neon-sjer-site",
                           "2013",
                           "insitu",
                           "veg_structure",
                           "D17_2013_SJER_vegStr.csv")

#https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
sjer_insitu_all = pd.read_csv(path_insitu) 

# View columns in data
sjer_insitu_all.columns

In [ ]:
sjer_insitu = sjer_insitu_all[["siteid",
                               "sitename",
                               "plotid",
                               "stemheight",
                               "scientificname"]]

sjer_insitu.head()

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
insitu_stem_ht = sjer_insitu.groupby('plotid').agg(
    ['mean', 'max'])['stemheight'] 

insitu_stem_ht.head()

In [ ]:
# Rename each column
insitu_stem_ht.head()

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
insitu_stem_ht.rename(columns={"mean": "insitu_mean",
                               "max": "insitu_max"},
                      inplace=True)  

insitu_stem_ht.head()

In [ ]:
# Reset the index (plotid)
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
insitu_stem_ht = insitu_stem_ht.reset_index() 
insitu_stem_ht.head()

In [ ]:
# Rename columns so that we know which columns represent lidar values
sjer_lidar_height_df = sjer_lidar_height_df.rename(
    columns={'max': 'lidar_max',
             'mean': 'lidar_mean',
             'min': 'lidar_min'})

# Join lidar and human measured tree height data
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
sjer_final_height = sjer_lidar_height_df.merge(insitu_stem_ht,
                                               left_on='Plot_ID',
                                               right_on='plotid') 
sjer_final_height.head()

In [ ]:
# Convert to a dataframe so you can use standard pandas plotting
sjer_final_height_df = pd.DataFrame(sjer_final_height)

fig, ax = plt.subplots(figsize=(10, 10))

sjer_final_height_df.plot('lidar_max',
                          'insitu_max',
                          kind='scatter',
                          fontsize=14, s=60,
                          color="purple",
                          ax=ax)

ax.set(xlabel="Lidar derived max tree height",
       ylabel="Measured tree height (m)",
       title="Lidar vs Measured Max Tree Height \n NEON SJER Field Site")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sjer_final_height_df.plot('lidar_max',
                          'insitu_max',
                          kind='scatter',
                          fontsize=14,
                          color="purple",
                          s=60, ax=ax)

ax.set(xlabel="Lidar Derived Max Tree Height (m)",
       ylabel="Measured Tree Height (m)",
       title="Lidar vs. Measured Max Tree Height \n NEON SJER Field Site")

# Add 1:1 line
ax.plot((0, 1), (0, 1),
        transform=ax.transAxes, ls='--', c='k')

# Adjust x and y axis limits
ax.set(xlim=[0, 30], ylim=[0, 30])
plt.show()

In [ ]:
# Export the final data frame as a csv file
outpath = os.path.join("spatial-vector-lidar",
                       "outputs",
                       "sjer-lidar-insitu-merge.csv")

sjer_final_height_df.to_csv(outpath)

In [ ]:
# Convert the geometry column to contain points
#https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.centroid.html
sjer_final_height['geometry'] = sjer_final_height.centroid 
sjer_final_height.head()

sjer_final_height['insitu_max']

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ep.plot_bands(sjer_chm_data,
              cmap='Greys',
              extent=plotting_extent(sjer_chm_data,
                                     sjer_chm_data.rio.transform()),
              ax=ax,
              scale=False)

# Plot centroids of each geometry as points so that you can control their size
sjer_final_height.centroid.plot(ax=ax,
                                marker='o',
                                markersize=sjer_final_height['insitu_max'] * 80,
                                c='purple')
ax.set_axis_off()
plt.show()

In [ ]:
# Calculate difference
sjer_final_height["lidar_measured"] = sjer_final_height["lidar_max"] - \
    sjer_final_height["insitu_max"]

# Create a bar plot
fig, ax = plt.subplots(figsize=(12, 7))
ax.bar(sjer_final_height['plotid'],
       sjer_final_height['lidar_measured'],
       color="purple")

ax.set(xlabel='Plot ID', ylabel='(Lidar - Measured) Height Difference (m)',
       title='Difference Between lidar and Measured Tree Height')

plt.setp(ax.get_xticklabels(),
         rotation=45, horizontalalignment='right')
plt.show()

## Part 3: Regression to Compare Variables

In [ ]:
#https://docs.python.org/3/library/math.html
from math import * 
#https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy import stats 


In [ ]:
x = sjer_final_height_df.lidar_max
y = sjer_final_height_df.insitu_max

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html#scipy.stats.linregress
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y) 

print("slope:", slope,
      "\nintercept:", intercept,
      "\nr squared:", r_value**2)

In [ ]:
# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 10))

m = slope.astype(float)

sjer_final_height_df.plot('lidar_max',
                          'insitu_max',
                          kind='scatter',
                          color="purple",
                          s=60,
                          ax=ax,
                          label="Data")

# Add a diagonal line
ax.set(xlim=[0, 30], ylim=[0, 30])
ax.plot((0, 1), (0, 1), 'y-', transform=ax.transAxes, label="1:1 line")
ax.plot(x, m*x + intercept, 'grey', label='regression fitted line')

ax.set(xlabel="Lidar derived max tree height (m)",
       ylabel="Measured tree height (m)",
       title="Lidar vs Measured Tree Height - SJER")

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

#https://seaborn.pydata.org/generated/seaborn.regplot.html
ax = sns.regplot(x='lidar_max', y='insitu_max', data=sjer_final_height_df,
                 color="purple") 

# Add a diagonal line
ax.set(xlim=[5, 30], ylim=[5, 30])
ax.plot((0, 1), (0, 1), transform=ax.transAxes, ls='--', c='k')

ax.set(xlabel="Lidar derived max tree height (m)",
       ylabel="Measured tree height (m)",
       title="Lidar vs Measured Tree Height - SJER")

plt.show()

In [ ]:
# Calculate difference and add the plot id to each xaxis label
sjer_final_height["lidar_measured"] = sjer_final_height["lidar_max"] - \
    sjer_final_height["insitu_max"]

fig, ax = plt.subplots(figsize=(12, 7))

ax.bar(sjer_final_height['plotid'],
       sjer_final_height['lidar_measured'],
       color="purple")

ax.set(xlabel='Plot ID',
       ylabel='(Lidar - Measured) Height Difference (m)',
       title='Difference Between lidar and Measured Tree Height')

plt.setp(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

## Part 4: Optional extra credit
1. Which site had the biggest discrepancy between insitu and lidar? Please specify the Plot ID. (1 point)

Tips:
* Use `sjer_final_height` GeoDataFrame
* Create a variable called `lidar_max` that valculates the maximum value in `lidar_measured` column
    * `sjer_final_height["lidar_measured"].max()`
* Subset `sjer_final_height` on this condition: `sjer_final_height["lidar_measured"] == lidar_max`
  * You can subset an GeoDataFrame by condition like this: `df[condition]`


2. What is the mean stemheight for *Arctostaphylos viscida* across all plots? (1 point)

Tips:
* Use `sjer_insitu` DataFrame
* Group by values in the `scientificname` column
  * You can groupby a column like this: `df.groupby("column")`
* Calculate the mean of each group
  * You can calculate the mean of a groupby argument like this: `df.groupby("column").mean(numeric_only=True)` 

3. Which plot ID only had a single species present?  (1 point)

Tips:
* Use `sjer_insitu` DataFrame
* Group by `plotid`
  * You can group a DataFrame by a column's values like this: `df.groupby("column")`
* Calculate the count of `scientificname` column on each group
  * You can calculate the count of a column's values in a group like this: `df.groupby("column")["another_column"].count()`

## Deliverable
1. Click File > Download > Download .ipynb
2. Navigate to the GitHub repository you created during Lab 0
3. Click Add file > Upload Files
4. Click choose your files and select the Lab 4 .ipynb file you downloaded
5. Click Commit changes
6. Navigate to the issue you opened in https://github.com/alex-pakalniskis/gisc606-spring2023 during lab 0 and leave a comment with a link to the Lab 4 .ipynb file you uploaded to your repo